In [1]:
from ont_fast5_api.fast5_interface import get_fast5_file
import pandas as pd
from Settings import *
from HelperFunctions import loadReads, fullNormalise, getMaxSim, getMinDist
import LSTMAutoencoder
import numpy as np
import time

Old testset:

1 - 00014eb4-4e2c-4087-ac24-57dea735b7b4

2 - 601f7bc7-3c09-4a78-a9b7-097bddcde809

3 - 1b6939ba-4a35-4696-bc6f-2ddb3368266e

4 - 95c03c50-57a5-4092-9a5f-87182d06a12c

5 - b6d8845b-3eec-42f6-9510-b8ba1fc1ec44


1 <--> 2    90.1

3 <--> 4    91.2

4 <--> 5    95.9 or 90.1

New testset:

9ee3897c-2329-4fc6-9ef3-d8eb97cd4919 --> 29fd6fb8-0f2f-4aa2-aa1a-c367f327acb6

f5e994e9-38a1-4079-adb5-bdacf43e0a17 --> 83e5572b-b883-4c88-a500-777144d48134

b61cb37c-ea80-4011-ac7a-6c853fd73eb6 --> 68b93e56-74fa-431b-9f40-a214a8bfc0b8

93ab5242-dab6-4609-b82c-04746416b6f1 --> 8edd07ab-777f-4784-ab0b-c268772d3113

edd987c6-0a3f-4214-a630-a1883b4f0f1a --> 7f7dfe85-7276-4a9d-ad84-2fa6dfa67635


In [3]:
squiggles, ids = loadReads()

reading squiggles file...


['29fd6fb8-0f2f-4aa2-aa1a-c367f327acb6',
 '68b93e56-74fa-431b-9f40-a214a8bfc0b8',
 '7f7dfe85-7276-4a9d-ad84-2fa6dfa67635',
 '83e5572b-b883-4c88-a500-777144d48134',
 '8edd07ab-777f-4784-ab0b-c268772d3113',
 '93ab5242-dab6-4609-b82c-04746416b6f1',
 '9ee3897c-2329-4fc6-9ef3-d8eb97cd4919',
 'b61cb37c-ea80-4011-ac7a-6c853fd73eb6',
 'edd987c6-0a3f-4214-a630-a1883b4f0f1a',
 'f5e994e9-38a1-4079-adb5-bdacf43e0a17']

In [4]:
for i in range(len(ids)):
    print(f"{ids[i]}: {len(squiggles[i])}")

29fd6fb8-0f2f-4aa2-aa1a-c367f327acb6: 255522
68b93e56-74fa-431b-9f40-a214a8bfc0b8: 192519
7f7dfe85-7276-4a9d-ad84-2fa6dfa67635: 165199
83e5572b-b883-4c88-a500-777144d48134: 195519
8edd07ab-777f-4784-ab0b-c268772d3113: 207479
93ab5242-dab6-4609-b82c-04746416b6f1: 113824
9ee3897c-2329-4fc6-9ef3-d8eb97cd4919: 98700
b61cb37c-ea80-4011-ac7a-6c853fd73eb6: 107332
edd987c6-0a3f-4214-a630-a1883b4f0f1a: 140733
f5e994e9-38a1-4079-adb5-bdacf43e0a17: 110049


In [4]:
model = LSTMAutoencoder.LSTM()
model.load_state_dict(torch.load(model_path))
model.eval()            #setting model to "predict mode"

LSTM(
  (lstm): LSTM(1, 30)
  (fc): Linear(in_features=30, out_features=1, bias=True)
)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
stateHs = []
stateCs = []
for i in range(len(ids)):
    print(f"embedding {i}...")
    s = time.time()
    x = fullNormalise(squiggles[i])

    x = torch.tensor(scaler.fit_transform(x.reshape(-1, 1))).squeeze(-1)
    x = x.type(torch.FloatTensor)

    state_hAcc = []
    state_cAcc = []

    chunks = x.unfold(dimension=0, size=sequenceLength, step=nGramStep)

    for chunkIndex in range(len(chunks)):#torch.split(x, sequenceLength):

        if chunkIndex % 1000 == 0:
            print(f"Chunk {chunkIndex}/{len(chunks)}")

        state_h = torch.zeros(1, hidden_size)
        state_c = torch.zeros(1, hidden_size)

        with torch.no_grad():
            for value in chunks[chunkIndex]:
                pred, (state_h, state_c) = model(value, (state_h, state_c))

        state_hAcc.append(state_h.squeeze(0))
        state_cAcc.append(state_c.squeeze(0))

        
    
    stateHs.append(state_hAcc)#stateHs.append(torch.mean(torch.stack(state_hAcc), dim=0))
    stateCs.append(state_cAcc)#stateCs.append(torch.mean(torch.stack(state_cAcc), dim=0))
    print(f"Took {time.time()-s}s.")

    

embedding 0...
Chunk 0/12758
Chunk 1000/12758
Chunk 2000/12758
Chunk 3000/12758
Chunk 4000/12758
Chunk 5000/12758
Chunk 6000/12758
Chunk 7000/12758
Chunk 8000/12758
Chunk 9000/12758
Chunk 10000/12758
Chunk 11000/12758
Chunk 12000/12758
Took 323.0412583351135s.
embedding 1...
Chunk 0/9615
Chunk 1000/9615
Chunk 2000/9615
Chunk 3000/9615
Chunk 4000/9615
Chunk 5000/9615
Chunk 6000/9615
Chunk 7000/9615
Chunk 8000/9615
Chunk 9000/9615
Took 192.60083413124084s.
embedding 2...
Chunk 0/8240
Chunk 1000/8240
Chunk 2000/8240
Chunk 3000/8240
Chunk 4000/8240
Chunk 5000/8240
Chunk 6000/8240
Chunk 7000/8240
Chunk 8000/8240
Took 176.0486137866974s.
embedding 3...
Chunk 0/9760
Chunk 1000/9760
Chunk 2000/9760
Chunk 3000/9760
Chunk 4000/9760
Chunk 5000/9760
Chunk 6000/9760
Chunk 7000/9760
Chunk 8000/9760
Chunk 9000/9760
Took 213.26341843605042s.
embedding 4...
Chunk 0/10364
Chunk 1000/10364
Chunk 2000/10364
Chunk 3000/10364
Chunk 4000/10364
Chunk 5000/10364
Chunk 6000/10364
Chunk 7000/10364
Chunk 8000/103

In [6]:
print(len(stateCs[0]))
print(len(squiggles[0]))
#print(stateCs[1])


12758
255522


In [9]:
#new similarity measure
#for each embeddings list in stateHs:
    #for each embedding:
        #for each embeddings list:
            #get the max similarity between current embedding and vectors in this list (and divide by length of list?)
            #add this max to the "similarities" measure of the current embeddings list
similartiesH = {}
similartiesC = {}

outerStart = time.time()
for el in range(len(stateHs)):
    print(ids[el])
    elStart = time.time()
    innerDictH = {}
    innerDictC = {}

    for innerel in range(len(stateHs)):
        if innerel != el:
            print(f"    comparing embeddings of squiggle {el} with embeddings of squiggle {innerel}")
            innerelStart = time.time()
            innerDictH[ids[innerel]] = 0
            innerDictC[ids[innerel]] = 0

            for e in range(len(stateHs[el])):
                if e % 1000 == 0:
                    print(f"        embedding {e}/{len(stateHs[el])}")
                innerDictH[ids[innerel]] += getMinDist(stateHs[el][e], stateHs[innerel])
                innerDictC[ids[innerel]] += getMinDist(stateCs[el][e], stateCs[innerel])

            print(f"    {el} to {innerel} comparisons took {time.time()-innerelStart}s.")
        
    similartiesH[ids[el]] = innerDictH.copy()
    similartiesC[ids[el]] = innerDictC.copy()

    print(f"All {ids[el]} comparisons took {time.time()-elStart}s.")

print(f"All comparisons took {time.time()-outerStart}s.")



29fd6fb8-0f2f-4aa2-aa1a-c367f327acb6
    comparing embeddings of squiggle 0 with embeddings of squiggle 1
        embedding 0/12758
        embedding 1000/12758
        embedding 2000/12758
        embedding 3000/12758
        embedding 4000/12758
        embedding 5000/12758
        embedding 6000/12758
        embedding 7000/12758
        embedding 8000/12758
        embedding 9000/12758
        embedding 10000/12758
        embedding 11000/12758
        embedding 12000/12758
    0 to 1 comparisons took 3271.177103281021s.
    comparing embeddings of squiggle 0 with embeddings of squiggle 2
        embedding 0/12758
        embedding 1000/12758
        embedding 2000/12758
        embedding 3000/12758
        embedding 4000/12758
        embedding 5000/12758
        embedding 6000/12758
        embedding 7000/12758
        embedding 8000/12758
        embedding 9000/12758
        embedding 10000/12758
        embedding 11000/12758
        embedding 12000/12758
    0 to 2 comparisons 

In [10]:
import json

with open("hSimilarties.json", "w") as file:
    json.dump(similartiesH, file)

with open("cSimilarties.json", "w") as file:
    json.dump(similartiesC, file)

In [11]:
for id in range(len(ids)):
    print(ids[id], len(squiggles[id]))

29fd6fb8-0f2f-4aa2-aa1a-c367f327acb6 255522
68b93e56-74fa-431b-9f40-a214a8bfc0b8 192519
7f7dfe85-7276-4a9d-ad84-2fa6dfa67635 165199
83e5572b-b883-4c88-a500-777144d48134 195519
8edd07ab-777f-4784-ab0b-c268772d3113 207479
93ab5242-dab6-4609-b82c-04746416b6f1 113824
9ee3897c-2329-4fc6-9ef3-d8eb97cd4919 98700
b61cb37c-ea80-4011-ac7a-6c853fd73eb6 107332
edd987c6-0a3f-4214-a630-a1883b4f0f1a 140733
f5e994e9-38a1-4079-adb5-bdacf43e0a17 110049
